# AA3 - Práctica 3

**Andrés Lires Saborido - andres.lires@udc.es**

### Descripción

En este ejercicio se va a crear un sistema de aprendizaje supervisado sobre el dataset CIFAR-100. Consta de las siguientes particiones:

- **train**: 50.000 ejemplos etiquetados en 100 clases
- **test**: 10.000 ejemplos etiquetados en 100 clases

De los datos de train, se deberán de eliminar el 80% de las etiquetas. Para ello usa el siguiente código:

```python
np.random.seed(125)
p = np.random.randn(len(train_label), 1)
unlabelled_data = train_dat[p > 0.2]
train_data = train_data[p <= 0.2]
train_label = train_label[p <= 0.2]
```

### Trabajo a realizar

1. Entrena un modelo, creado sobre Tensorflow, haciendo uso únicamente de los datos etiquetados en train. Dicho modelo tiene que ser enteramente convolucional, a excepción de la última capa. El número de capas convolucionales no será inferior a 4. Responde a las siguientes preguntas:

    - ¿Qué red has escogido? ¿Cómo la has entrenado?
    - ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?
    - ¿Qué conclusiones sacas de los resultados del punto anterior?


2. Entrena el mismo modelo mediante la técnica de auto aprendizaje. Será obligatorio ponderar cada ejemplo de entrada en función de su calidad (o certeza). Responde a las siguientes preguntas:

    - ¿Qué parámetros has definido para el entrenamiento?
    - ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?
    - ¿Se mejoran los resultados obtenidos en el ejercicio anterior?
    - ¿Qué conclusiones sacas de los resultados del punto anterior?

3. Entrena un modelo de aprendizaje semisupervisado de tipo autoencoder en dos pasos (primero el autoencoder, después el clasificador). Se cumplirán los siguientes requisitos:

    - La arquitectura del encoder tiene que ser exactamente la misma a la utilizada en el clasificador definido anteriormente en el ejercicio 1, a excepción la última capa y de la capa anterior de Pooling (si existe).
    - Se prohíbe el uso de capas Dense en el decoder, esto es, el autoencoder será totalmente convolucional.

    Responde a las siguientes preguntas:

    1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?
    2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?
    3. ¿Se mejoran los resultados obtenidos en los puntos anteriores?
    4. ¿Qué conclusiones sacas de este apartado?

4. Entrena un modelo de aprendizaje semisupervisado de tipo autoencoder en un paso (autoencoder y clasificador al mismo tiempo). Se cumplirán los siguientes puntos:

    - La arquitectura del autoencoder será exactamente la misma del punto 3.
    - El resultado de combinar el encoder con el clasificador será exactamente igual a la arquitectura definida en el punto 1.

    Responde a las siguientes preguntas:

    1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?
    2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?
    3. ¿Se mejoran los resultados obtenidos en los puntos anteriores?
    4. ¿Qué conclusiones sacas de este apartado?

5. Repite el mismo entrenamiento de los 4 ejercicios anteriores, pero eliminando los datos sin etiquetar más atípicos con respecto a los datos etiquetados. Se cumplirán los siguientes puntos:

    - La arquitectura de la red one-class tiene que ser exactamente la misma a la utilizada en el clasificador definido anteriormente en el ejercicio 1, a excepción de la capa de salida.
    - Utiliza la técnica explicada en el lab9, usando un valor de ν=0.9.

    Responde a la siguiente pregunta: ¿Se mejoran los resultados?

6. Repite los puntos 3, 4 y 5, pero cambiando el autoencoder por la técnica definida en el apartado "Hay vida más allá del autoencoder" del lab8 de prácticas. Contesta a las preguntas de dichos puntos. Se cumplirán los siguientes puntos:

    - La arquitectura de la red será igual a la parte del encoder del autoencoder que hayas definido en los puntos anteriores.
    - El modelo tiene que entrenar correctamente.

### Resolución práctica

#### 1. Carga de datos

Importamos las librerías necesarias:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt

Cargamos el dataset CIFAR-100:

In [2]:
(x_train, y_train), (x_test, y_test), = tf.keras.datasets.cifar100.load_data()

Comprobamos que el dataset se ha cargado correctamente:

In [3]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

assert len(np.unique(y_train)) == 100

Preprocesamos los datos para que estén en el rango [0, 1] y eliminamos el 80% de las etiquetas:

In [4]:
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

# Normalizamos los datos
x_train = x_train / 255.0
x_test = x_test / 255.0

# Eliminamos el 80% de las etiquetas
np.random.seed(125)
p = np.random.rand(len(y_train))
unlabelled_data = x_train[p > 0.2]
train_data = x_train[p <= 0.2]
train_label = y_train[p <= 0.2]

In [5]:
# Comprobamos las dimensiones
print("Dimensiones de train_data", train_data.shape)
print("Dimensiones de unlabelled_data:", unlabelled_data.shape)
print("Dimensiones de label_data:", train_label.shape)
print("Número de clases:", len(np.unique(train_label)))

# Constantes
INPUT_SHAPE = (32, 32, 3)
NUM_CLASES = len(np.unique(train_label))

Dimensiones de train_data (10080, 32, 32, 3)
Dimensiones de unlabelled_data: (39920, 32, 32, 3)
Dimensiones de label_data: (10080,)
Número de clases: 100


### Modelo 1: Modelo convolucional sobre datos etiquetados

In [ ]:
NUM_CLASES = len(np.unique(train_label))
INPUT_SHAPE = (32, 32, 3)

class ModeloCNNEtiquetados:

    def __init__(self, input_shape, num_clases):

        input_layer = layers.Input(shape=input_shape)
        x = layers.Conv2D(16, (3, 3), activation='relu')(input_layer)
        x = layers.Conv2D(32, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(128, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Flatten()(x)
        output_layer = layers.Dense(num_clases, activation='softmax')(x)

        self.model = models.Model(inputs=input_layer, outputs=output_layer)

        self.model.compile(optimizer=optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

    def fit(self, X, y, batch_size=128 , epochs=100, sample_weight=None):
        self.model.fit(X, y, batch_size=batch_size, epochs=epochs, sample_weight=sample_weight)

    def summary(self):
        self.model.summary()

    def predict(self, X):
        pred = self.model.predict(X)
        pred = np.argmax(pred, axis=1)
        return pred

    def predict_proba(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        loss, acc = self.model.evaluate(X, y, verbose=0)
        return acc

    def __del__(self):
        del self.model
        tf.keras.backend.clear_session()

In [ ]:
# Creamos el modelo
model = ModeloCNNEtiquetados(INPUT_SHAPE, NUM_CLASES)
model.summary()

history = model.fit(train_data, train_label,
                    epochs=100, batch_size=128)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        51,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 148,740 (581.02 KB)

 Trainable params: 148,740 (581.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.0138 - loss: 4.5776
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.0458 - loss: 4.2471
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1123 - loss: 3.8704
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1412 - loss: 3.6189
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1792 - loss: 3.4252
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2115 - loss: 3.2621
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2506 - loss: 3.0986
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2662 - loss: 2.9743
Epoch 9/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2877 - loss: 2.8539
Epoch 10/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3134 - loss: 2.7381
Epoch 11/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3344 - loss: 2.6328
Epoch 12/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy

Evaluamos en el conjunto de train y en el conjunto de test:

In [ ]:
# Evaluamos en train
train_acc = model.score(train_data, train_label)
print('Train accuracy:', round(train_acc, 5))

# Evaluamos en test
test_acc = model.score(x_test, y_test)
print('Test accuracy:', round(test_acc, 5))

Train accuracy: 0.99296
Test accuracy: 0.2279


**<u>Cuestiones:</u>**


1. **¿Qué red has escogido? ¿Cómo la has entrenado?**

    Creamos un modelo convolucional de 4 capas *Conv2D* y 3 capas de pooling. Las capas convolucionales de 16, 32, 64 y 128 neuronas respectivamente, aumentan el número de canales mientras reducen el tamaño de estes y cada una de las capas de *MaxPooling* nos ayudará a reducir el número de parámetros entrenables. Por último se aplica una capa *flatten* y se obtiene la salida predicha con una *Dense* de 100 neuronas (una por cada clase)
    con función de activación *softmax*.

    Para el entrenamiento se utilizan los conjuntos de *train_data* y *train_label*s, con un tamaño de batch de 128 y 100 epochs. La función de pérdida utilizada es *categorical_crossentropy* y el optimizador *adam*.

2. **¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?**
    
    La precisión en entrenamiento es de $0.99296$ mientras que en el conjunto de test es de $0.2279$.

3. **¿Qué conclusiones sacas de los resultados del punto anterior?**

    Está claro que el modelo esta sobreentrenando. Esto seguramente se deba a que estamos utilizando una red muy compleja para un conjunto de datos reducido (recordamos que solo estamos utilizando el 20% etiquetado de los datos originales). Nuestro modelo no está generalizando bien.

### Modelo 2: Autoaprendizaje

Implementamos el algoritmo de autoaprendizaje:

- **self_training** *(model, x_train, y_train, unlabeled_data, thresh, train_epochs)*

    1. $train\_data, train\_label \leftarrow x\_train, y\_train$
    1. **Desde** $n = 1 .. train\_epochs$ **hacer**
        1. Instancia un nuevo *model* y entrénalo usando las variables *train\_data* y *train\_label*
        2. $y\_pred \leftarrow model(unlabeled\_data)$
        3. $y\_class, y\_value \leftarrow $ Clase ganadora en *y_pred* con su valor
        4. $train\_data, train\_label \leftarrow x\_train, y\_train$
        5. **Para cada elemento** (x_u, y_c, y_v) **de la tupla** ($unlabeled\_data$, $y\_class$, $y\_value$)
            1. **Si** $y\_v > thresh$ **entonces**
                1. Añadimos $x\_u$ e $y\_c$ a $train\_data$ y $train\_label$, respectivamente.
    4. Devolvemos el modelo

In [ ]:
def self_training_v2(model_creator, x_train, y_train, unlabeled_data, x_test, y_test, thresh=0.8, train_epochs=3):
    # Creamos una copia de los datos no etiquetados para no modificar el original
    train_data = x_train.copy()
    train_label = y_train.copy()

    # Pesos iniciales para los datos etiquetados
    weights = np.ones(len(train_label))

    for i in range(train_epochs):
        model = model_creator()
        if i==0:
            model.summary()
        # Entrenamos el modelo con los datos etiquetados y sus pesos
        model.fit(train_data, train_label, sample_weight=weights)

        # Predecimos las probabilidades en el conjunto no etiquetado
        y_pred = model.predict_proba(unlabeled_data)
        y_class = np.argmax(y_pred, axis=1)
        y_value = np.max(y_pred, axis=1)

        # Seleccionamos solo aquellos con probabilidad mayor o igual al umbral
        indexes = np.where(y_value >= thresh)[0]

        # Añadimos las muestras seleccionadas al conjunto de entrenamiento
        new_data = unlabeled_data[indexes]
        new_labels = y_class[indexes]

        # Peso proporcional a la certeza de la predicción
        new_weights = y_value[indexes]

        train_data = np.concatenate((train_data, new_data))
        train_label = np.concatenate((train_label, new_labels))
        weights = np.concatenate((weights, new_weights))

        # Eliminamos los datos etiquetados del conjunto no etiquetado
        unlabeled_data = np.delete(unlabeled_data, indexes, axis=0)

        # Evaluamos en el conjunto de test
        print("Epoch:", i+1, "Accuracy:", model.score(x_test, y_test))
        print("Número de elementos en el conjunto etiquetado:", len(train_label))

    return model

Utilizamos el modelo clasificador anterior:

In [ ]:
model_func = lambda: ModeloCNNEtiquetados(INPUT_SHAPE, NUM_CLASES)

Entrenamos el modelo de autoaprendizaje:

In [ ]:
model = self_training_v2(model_func, train_data, train_label, unlabelled_data, x_test, y_test, train_epochs=3)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        51,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 148,740 (581.02 KB)

 Trainable params: 148,740 (581.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.0164 - loss: 4.5457
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0531 - loss: 4.1922
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1087 - loss: 3.8750
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1520 - loss: 3.6110
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1883 - loss: 3.4273
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2209 - loss: 3.2400
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2471 - loss: 3.1254
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2744 - loss: 2.9387
Epoch 9/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2927 - loss: 2.8608
Epoch 10/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3277 - loss: 2.6794
Epoch 11/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3525 - loss: 2.5675
Epoch 12/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy

Evaluamos en el conjunto de entrenamiento y en el de test:

In [ ]:
# Evaluamos en train
train_acc = model.score(train_data, train_label)
print('Train accuracy:', round(train_acc, 5))

# Evaluamos en test
test_acc = model.score(x_test, y_test)
print('Test accuracy:', round(test_acc, 5))

Train accuracy: 0.91121
Test accuracy: 0.2165


    
- **<u>Cuestiones:</u>**

    1. **¿Qué parámetros has definido para el entrenamiento?**

        - *train\_epochs*: 3 (Número de épocas para el self-training)
        - *thresh*: 0.8 (Umbral de confianza para añadir ejemplos no etiquetados al conjunto de entrenamiento)
        - *batch\_size*: 128 (Tamaño del batch para el entrenamiento del modelo)
        - *epochs*: 100 (Número de épocas para el entrenamiento del modelo)

          Al igual que en el modelo anterior, la función de pérdida utilizada es *categorical_crossentropy* y el optimizador *adam*.

          Además se utiliza un vector de pesos para ponderar cada ejemplo de entrada en función de su calidad (o certeza). Este vector será 1 para los ejemplos etiquetados y se establecerá según su probabilidad o certeza de pertenencia a la clase para los ejemplos no etiquetados que sean añadidos al conjunto de entrenamiento.

    2. **¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?**

        La precisión en entrenamiento es de $0.91121$ mientras que en el conjunto de test es de $0.2165$.
    
        A la vista de los resultados, el modelo no ha mejorado su precisión en el conjunto de test, aunque sí ha aumentado la cantidad de datos etiquetados.

    3. **¿Se mejoran los resultados obtenidos en el ejercicio anterior?**

        A la vista de los resultados, las mejoras del modelo de autoaprendizaje no son significativas.

    4. **¿Qué conclusiones sacas de los resultados del punto anterior?**

        Estamos añadiendo patrones al conjunto de entrenamiento, por lo que reducimos el problema de que nuestro modelo se ajuste para unos pocos datos concretos, sin embargo, a la vista de los resultados de test seguimos sin lograr una buena generalización.


### Modelo 3: Autoencoder en dos pasos

Definimos el autoencoder:

In [ ]:
class MiAutoencoder:

    def __init__(self, input_shape):
        # Input layer
        input_layer = layers.Input(shape=input_shape)

        # Encoder
        x = layers.Conv2D(16, (3, 3), activation='relu')(input_layer)
        x = layers.Conv2D(32, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        encoded = layers.Conv2D(128, (3, 3), activation='relu')(x)

        # Decoder
        x = layers.Conv2DTranspose(128, (3, 3), activation='relu')(encoded)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(64, (3, 3), activation='relu')(x)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(32, (3, 3), activation='relu')(x)
        x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
        decoded = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid')(x)

        self.encoder = models.Model(inputs=input_layer, outputs=encoded)

        self.autoencoder = models.Model(inputs=input_layer, outputs=decoded)

        self.autoencoder.compile(optimizer=optimizers.Adam(),
                                   loss=tf.keras.losses.MeanSquaredError())


    def fit(self, X, y=None, sample_weight=None):
        # Entrenar el autoencoder para que aprenda a reconstruir
        self.autoencoder.fit(X, X, batch_size=128, epochs=50, sample_weight=sample_weight)

    def summary(self):
        self.autoencoder.summary()

    def get_encoded_data(self, X):
        # Obtener la representación codificada (output del encoder)
        return self.encoder.predict(X)

    def __del__(self):
        # Elimina todos los modelos que hayas creado
        tf.keras.backend.clear_session()  # Necesario para liberar la memoria en GPU


Definimos el clasificador:

In [12]:
class MiClasificador:

    def __init__(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(shape=(4, 4, 128)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(NUM_CLASES, activation='softmax')
        ])

        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def fit(self, X, y, sample_weight=None):
        # Entrenar el modelo
        self.model.fit(X, y, batch_size=256, epochs=100, sample_weight=sample_weight) # Cambiado a 256 para el ej.6

    def predict(self, X):
        # Predecir la clase ganadora
        return np.argmax(self.model.predict(X), axis=1)

    def predict_proba(self, X):
        # Predecir probabilidades de clase
        return self.model.predict(X)

    def score(self, X, y):
        # Evaluar el modelo
        _, acc = self.model.evaluate(X, y, verbose=0)
        return acc

Definimos la función del entrenamiento semisupervisado en 2 pasos:

In [7]:
def semisupervised_training(autoencoder, classifier, x_train, y_train, unlabeled_data):
    # Concatenar los datos y entrenar el autoencoder
    all_data = np.concatenate([x_train, unlabeled_data], axis=0)
    autoencoder.fit(all_data)
    encoded_data = autoencoder.get_encoded_data(x_train)

    # Entrenar el clasificador simple
    classifier.fit(encoded_data, y_train)

    return autoencoder, classifier


Entrenamos nuestro modelo:

In [ ]:
# Crea tu autoencoder y tu clasificador
autoencoder = MiAutoencoder(input_shape=INPUT_SHAPE)
autoencoder.summary()

classifier = MiClasificador()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 6, 6, 128)      │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 14, 14, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 30, 30, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 30, 30, 16)     │         4,624 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 32, 32, 3)      │           435 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 342,339 (1.31 MB)

 Trainable params: 342,339 (1.31 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
semisupervised_training(autoencoder, classifier, train_data, train_label, unlabelled_data)

Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - loss: 0.0324
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0097
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0078
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0071
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0067
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0062
Epoch 7/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0059
Epoch 8/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0056
Epoch 9/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0055
Epoch 10/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0052
Epoch 11/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0051
Epoch 12/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0050
Epoch 13/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0048
Epoch 14/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0047
Epoch 15/50
391/391 ━━━━━━━━━━━━━━━

(<__main__.MiAutoencoder at 0x7ba6999c5910>,
 <__main__.MiClasificador at 0x7ba754780290>)

Evaluamos el modelo:

In [ ]:
# Evaluamos en test
pred_test = autoencoder.get_encoded_data(x_test)
print('Test accuracy :', classifier.score(pred_test, y_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Test accuracy : 0.19290000200271606


- <u>Cuestiones</u>

  1. **¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?**

    El modelo consta de 2 partes, un autoencoder (encoder + decoder) y un clasificador.
    
    La arquitectura del encoder es exactamente la misma que la del modelo convolucional del primer ejercicio, a excepción de la última capa de MaxPooling, la capa de aplanamiento y la capa de salida. El decoder consta de 5 capas *Conv2DTranspose* y 2 capas de *UpSampling2D* que nos ayudarán a recuperar la imagen original.

    En cuanto al clasificador, se incluye la capa de entrada con las dimensiones de salida del encoder, una capa de *flatten* y 4 capas *Dense*, siendo la última de 100 neuronas (una por cada clase) con función de activación *softmax*.

    Los hiperparámetros utilizados son los siguientes:
    - *batch\_size*: 128
    - *epochs*: 50 en el autoencoder y 100 en el clasificador
    
    El optimizador utilizado es *adam* y la función de pérdida *categorical_crossentropy*.

  2. **¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?**

    La precisión durante el entrenamiento sube hasta $0.9490$. En el conjunto de test la precisión es de $0,1929$

  3. **¿Se mejoran los resultados obtenidos en los puntos anteriores?**

    La precisión en el conjunto de test ha bajado respecto al modelo de autoaprendizaje, aunque la precisión en el conjunto de entrenamiento ha aumentado.

  4. **¿Qué conclusiones sacas de este apartado?**

    El entrenamiento sigue teniendo problemas importantes de generalización. La precisión en el conjunto de test es muy baja, lo que indica que el modelo no está aprendiendo patrones generales de los datos. Esto puede deberse a la complejidad del modelo y a la cantidad limitada de datos etiquetados.

### Modelo 4: Autoencoder en un paso

Definimos un autoencoder con exactamente la misma arquitectura que en el apartado anterior.

El resultado de combinar el encoder con el clasificador será exactamente igual a la arquitectura definida en el punto 1.

In [ ]:
class MiClasificadorSemisupervisado:
    def __init__(self, input_shape):
        # Capa de entrada con input_shape aplanada 32*32*3
        input_layer = tf.keras.layers.Input(shape=input_shape)

        # Encoder
        # Recuperamos el tamaño (32, 32, 3)
        encoder = tf.keras.layers.Reshape((32, 32, 3))(input_layer)
        encoder = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(encoder)
        encoder = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(encoder)
        encoder = tf.keras.layers.MaxPooling2D((2, 2))(encoder)
        encoder = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(encoder)
        encoder = tf.keras.layers.MaxPooling2D((2, 2))(encoder)
        encoder = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(encoder)

        # Decoder
        decoder = layers.Conv2DTranspose(128, (3, 3), activation='relu')(encoder)
        decoder = layers.UpSampling2D((2, 2))(decoder)
        decoder = layers.Conv2DTranspose(64, (3, 3), activation='relu')(decoder)
        decoder = layers.UpSampling2D((2, 2))(decoder)
        decoder = layers.Conv2DTranspose(32, (3, 3), activation='relu')(decoder)
        decoder = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(decoder)
        decoder = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid')(decoder)

        decoder = tf.keras.layers.Flatten()(decoder)

        # Classifier
        classifier = tf.keras.layers.Flatten()(encoder)
        # Misma arquitectura que modelo 1, es decir una sola capa densa
        classifier = tf.keras.layers.Dense(100, activation='softmax')(classifier)

        # Creación del modelo
        self.model = tf.keras.models.Model(inputs=input_layer, outputs=[decoder, classifier])

        # Compilar
        self.model.compile(optimizer='adam', loss=['mse', 'sparse_categorical_crossentropy'], metrics=['accuracy', 'accuracy'])

    def fit(self, X, y, unlabeled_data):
        # Combinamos labeled y unlabeled data (con etiqueta 0)
        x_train_combined = np.concatenate([X, unlabeled_data], axis=0)
        y_train_combined = np.concatenate((y, np.full(len(unlabeled_data), 0)), axis=0)

        # Hacemos un aplanado
        x_train_combined = x_train_combined.reshape(x_train_combined.shape[0], -1)

        # Sample weights
        # Para el autoencoder todo 1 y para el clasificador sin tener en cuenta los no etiquetados
        sample_weight = [np.ones(len(x_train_combined)), np.concatenate((np.ones(len(y)), np.zeros(len(unlabeled_data))))]

        self.model.fit(
            x_train_combined,
            [x_train_combined, y_train_combined],
            batch_size=128,
            epochs=100,
            sample_weight = sample_weight,
            verbose=1
        )

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=1)

    def predict_proba(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        score = self.model.evaluate(X, [X, y])
        return score

    def __del__(self):
        tf.keras.backend.clear_session()

Redefinimos la función de semisupervised_training

In [ ]:
def semisupervised_training_v2(model, x_train, y_train, unlabeled_data):
    model.fit(x_train, y_train, unlabeled_data)

Entrenamos el modelo

In [ ]:
# Creamos el clasificador
model = MiClasificadorSemisupervisado(input_shape=(32*32*3, ))
semisupervised_training_v2(model, train_data, train_label, unlabelled_data)

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - dense_accuracy: 0.0081 - dense_loss: 0.8893 - flatten_accuracy: 0.0025 - flatten_loss: 0.0420 - loss: 0.9313
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - dense_accuracy: 0.0410 - dense_loss: 0.7373 - flatten_accuracy: 0.0029 - flatten_loss: 0.0241 - loss: 0.7614
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - dense_accuracy: 0.0531 - dense_loss: 0.6527 - flatten_accuracy: 0.0039 - flatten_loss: 0.0203 - loss: 0.6730
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - dense_accuracy: 0.0692 - dense_loss: 0.5634 - flatten_accuracy: 0.0044 - flatten_loss: 0.0186 - loss: 0.5820
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - dense_accuracy: 0.0843 - dense_loss: 0.4855 - flatten_accuracy: 0.0050 - flatten_loss: 0.0185 - loss: 0.5040
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - dense_accuracy: 0.1023 - dense_loss: 0.4141 - flatten_accuracy: 0.0058 - flatten_loss: 0.0183 - loss: 0.4324
Epoch 7/

Evaluamos en el conjunto de test

In [ ]:
x_test = x_test.reshape(x_test.shape[0], (32*32*3))
print('Test accuracy :', model.score(x_test, y_test)[3])

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - dense_accuracy: 0.2062 - dense_loss: 19.9344 - flatten_accuracy: 0.0082 - flatten_loss: 0.0159 - loss: 19.9502
Test accuracy : 0.2029999941587448


<u>Cuestiones</u>
1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?

  En este caso, tenemos un solo modelo con dos salidas diferentes que se entrenaran de forma paralela (autoencoder y encoder+clasificador)

  La arquitectura del autoencoder y del clasificador son las mismas que anteriormente.

  A la hora de compliar el modelo es importante destacar que utilizaremos diferentes funciones de pérdida para las salidas, *mse* y *sparse_categorical_crossentropy*.

  Los hiperparámetros son:
  - *batch\_size*: 128
  - *epochs*: 100


2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?

  En entrenamiento se alcanza una precisión de $0.2103$ y en el conjunto de test de $0.2029$.

3. ¿Se mejoran los resultados obtenidos en los puntos anteriores?

  Obtenemos el peor resultado hasta ahora en el entrenamiento pero el rendimiento en el conjunto de test sigue siendo similar.

4. ¿Qué conclusiones sacas de este apartado?

  El utilizar datos no etiquetados puede estar provocando que en el entrenamiento del autoencoder no se obtenga una buena representación en espacio reducido.

### Entrenamiento de los modelos eliminando datos atípicos

Reutilizamos código del lab9

In [17]:
class ChangeRCallback(tf.keras.callbacks.Callback):
   def __init__(self, train_data, delta=.025, steps=3):
       super().__init__()
       self.train_data = train_data
       self.delta = delta
       self.steps = steps
       self.cont = 0

   def on_epoch_end(self, epoch, logs=None):
       sorted_values = np.sort(self.model.predict(self.train_data).flatten())
       new_value = sorted_values[int(len(sorted_values) * (1. - self.model.nu))]
       old_value = self.model.r.numpy()
       print('Cambiando r a', new_value, ', max:', sorted_values.max(), ', min:', sorted_values.min())
       self.model.r.assign(new_value)
       if np.abs(old_value - new_value) < self.delta:
            self.cont += 1
            if self.cont >= self.steps:
                print('Convergencia obtenida. Finalizando el entrenamiento.')
                self.model.stop_training = True
       else:
            self.cont = 0

In [18]:
class DetectorAnomalias:

    def __init__(self, input_shape, nu=.5):
        # Todas las capas deben incluir regularización L2
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.InputLayer(shape=input_shape))
        self.model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        self.model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
        self.model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
        self.model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

        self.model.r = tf.Variable(1.0, trainable=False, name='r', dtype=tf.float32)
        self.model.nu = tf.Variable(nu, trainable=False, name='nu', dtype=tf.float32)

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        self.model.compile(optimizer=self.optimizer, loss=self.loss_function, metrics=['accuracy'])

    def loss_function(self, y_true, y_pred):
        loss = (1/self.model.nu) * tf.reduce_mean(tf.maximum(0., self.model.r - y_pred))
        return loss

    def fit(self, X, y=None, sample_weight=None):
        dummy_y = np.zeros((len(X), 1)) # Necesario pasar como salida para que keras no de un error
        self.model.fit(X, dummy_y, batch_size=128, epochs=100, verbose=1, callbacks=[ChangeRCallback(X, delta=0.025, steps=3)])

    def predict(self, X):
        pred = self.model.predict(X, batch_size=128, verbose=1)
        pred = np.where(pred > self.model.r, 1, 0)
        return pred

    def __del__(self):
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

Creamos el modelo y lo entrenamos

In [19]:
detector = DetectorAnomalias(input_shape=INPUT_SHAPE, nu=0.9)
detector.fit(train_data, y=None, sample_weight=None)

Epoch 1/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Cambiando r a 0.99590814 , max: 0.99999416 , min: 0.87893873
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - accuracy: 0.0621 - loss: 1.0552
Epoch 2/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Cambiando r a 0.99488777 , max: 0.99998486 , min: 0.8978026
79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.0000e+00 - loss: 0.1003
Epoch 3/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Cambiando r a 0.99517775 , max: 0.9999579 , min: 0.94331086
Convergencia obtenida. Finalizando el entrenamiento.
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 0.0412


In [20]:
atipicos = detector.predict(unlabelled_data)
idx = atipicos==1
print(f'Se han detectado como atipicos el: {100 - np.mean(atipicos.flatten())*100}% de los datos sin etiquetar')

312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Se han detectado como atipicos el: 10.36573146292585% de los datos sin etiquetar


In [21]:
unlabelled_data_sin_atipicos = unlabelled_data[idx.flatten(),:,:,:]

print('Tamaño de unlabeled_data', unlabelled_data.shape[0])
print('Tamaño de unlabeled_data_sin_atipicos:', unlabelled_data_sin_atipicos.shape[0], '(Eliminados', unlabelled_data.shape[0] - unlabelled_data_sin_atipicos.shape[0], ')')

Tamaño de unlabeled_data 39920
Tamaño de unlabeled_data_sin_atipicos: 35782 (Eliminados 4138 )


1. Modelo 1

In [ ]:
# Es exactamente lo mismo. No entrenamos de nuevo porque en este modelo no se utilizan los datos no etiquetados.

2. Modelo 2

In [ ]:
model = self_training_v2(model_func, train_data, train_label, unlabelled_data_sin_atipicos, x_test, y_test, train_epochs=3)

Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        51,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 148,740 (581.02 KB)

 Trainable params: 148,740 (581.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.0180 - loss: 4.5552
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0773 - loss: 4.1110
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1224 - loss: 3.8096
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1602 - loss: 3.5538
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1997 - loss: 3.3604
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2437 - loss: 3.1467
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2699 - loss: 2.9734
Epoch 8/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3010 - loss: 2.8203
Epoch 9/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3260 - loss: 2.7119
Epoch 10/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3490 - loss: 2.5886
Epoch 11/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3731 - loss: 2.4763
Epoch 12/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy

Exception ignored in: <function DetectorAnomalias.__del__ at 0x7e38f68bb240>
Traceback (most recent call last):
  File "<ipython-input-21-0e7cffeea22a>", line 39, in __del__
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/common/global_state.py", line 82, in clear_session
    tf.compat.v1.reset_default_graph()
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/ops.py", line 5111, in reset_default_graph
    raise AssertionError("Do not use tf.reset_default_graph() to clear "
AssertionError: Do not use tf.reset_default_graph() to clear nested graphs. If you need a cleared graph, exit the nesting and create a new graph.
Exception ignored in: <function DetectorAnomalias.__del__ at 0x7e3908cda5c0>
Traceback (most recent call last):
  File "<ipython-input-24-0e7cffeea22a>", line 39, in __del__
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/common/global_state.py", line 82, in clear_session
    tf.compat.v1.reset_default_graph()
  

Epoch: 1 Accuracy: 0.2295999974012375
Número de elementos en el conjunto etiquetado: 35478
Epoch 1/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.0545 - loss: 4.1572
Epoch 2/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2105 - loss: 3.2136
Epoch 3/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2830 - loss: 2.8164
Epoch 4/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3249 - loss: 2.5678
Epoch 5/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3615 - loss: 2.3816
Epoch 6/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3902 - loss: 2.2466
Epoch 7/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4212 - loss: 2.1198
Epoch 8/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4368 - loss: 2.0329
Epoch 9/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4579 - loss: 1.9309
Epoch 10/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4803 - loss: 1.8450
Epoch 11/100
278/278 ━━━━━━━━━━━━━━━━

In [ ]:
# Evaluamos en test
test_acc = model.score(x_test, y_test)
print('Test accuracy:', round(test_acc, 5))

Test accuracy: 0.22


3. Modelo 3

In [ ]:
semisupervised_training(autoencoder, classifier, train_data, train_label, unlabelled_data_sin_atipicos)

Epoch 1/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - loss: 0.0330
Epoch 2/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 0.0108
Epoch 3/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0083
Epoch 4/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0074
Epoch 5/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0067
Epoch 6/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 0.0063
Epoch 7/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0060
Epoch 8/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.0058
Epoch 9/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0056
Epoch 10/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.0054
Epoch 11/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0052
Epoch 12/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0052
Epoch 13/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0052
Epoch 14/50
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0050
Epoch 15/50
359/359 ━━━━━━━━━━━━━━━━━

(<__main__.MiAutoencoder at 0x7e390b0e60d0>,
 <__main__.MiClasificador at 0x7e39094180d0>)

In [ ]:
# Evaluamos en test
pred_test = autoencoder.get_encoded_data(x_test)
print('Test accuracy :', classifier.score(pred_test, y_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Test accuracy : 0.19519999623298645


4. Modelo 4

In [ ]:
model = MiClasificadorSemisupervisado(input_shape=(32*32*3, ))
semisupervised_training_v2(model, train_data, train_label, unlabelled_data_sin_atipicos)

Epoch 1/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - dense_accuracy: 0.0071 - dense_loss: 0.9707 - flatten_accuracy: 0.0013 - flatten_loss: 0.0122 - loss: 0.9829
Epoch 2/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - dense_accuracy: 0.0427 - dense_loss: 0.8221 - flatten_accuracy: 0.0028 - flatten_loss: 0.0070 - loss: 0.8291
Epoch 3/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - dense_accuracy: 0.0518 - dense_loss: 0.7247 - flatten_accuracy: 0.0025 - flatten_loss: 0.0062 - loss: 0.7310
Epoch 4/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - dense_accuracy: 0.0674 - dense_loss: 0.6582 - flatten_accuracy: 0.0036 - flatten_loss: 0.0056 - loss: 0.6638
Epoch 5/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - dense_accuracy: 0.0782 - dense_loss: 0.5992 - flatten_accuracy: 0.0037 - flatten_loss: 0.0053 - loss: 0.6045
Epoch 6/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - dense_accuracy: 0.0962 - dense_loss: 0.5144 - flatten_accuracy: 0.0042 - flatten_loss: 0.0050 - loss: 0.5194
Epoch 7/1

In [ ]:
x_test = x_test.reshape(x_test.shape[0], (32*32*3))
print('Test accuracy :', model.score(x_test, y_test)[3])

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - dense_accuracy: 0.1943 - dense_loss: 18.8466 - flatten_accuracy: 0.0077 - flatten_loss: 0.0241 - loss: 18.8707
Test accuracy : 0.19550000131130219


<u>Responde a la siguiente pregunta:</u>

1. ¿Se mejoran los resultados?

  - En el modelo 2, se mantiene más o menos igual (0.22)

  - En el modelo 3, la precisión es de 0.19, igual que antes de eliminar atípicos.

  - En el modelo 3, baja la precisión en 0.01

  No se aprecian mejoras significativas

### Cambiamos el Autoencoder por otra técnica

In [13]:
class ContrastiveLearning(tf.keras.Model):
    def __init__(self, input_shape, num_classes, lamda, tau):
        super(ContrastiveLearning, self).__init__()

        self.tau = tau
        self.lamda = lamda
        self.loss_tracker = tf.keras.metrics.Mean(name='loss')

        # Data augmentation
        self.data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomRotation(0.02),
            tf.keras.layers.RandomTranslation(0.15, 0.15),
            tf.keras.layers.RandomZoom(0.15)
        ])

        # Encoder
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        ])

        # Clasificador
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    @property
    def metrics(self):
        return [self.loss_tracker]

    def train_step(self, data):
        augmented_1, augmented_2 = self.data_augmentation(data), self.data_augmentation(data)

        with tf.GradientTape() as tape:

            # Encoder
            z_1 = self.encoder(augmented_1, training=True)
            z_2 = self.encoder(augmented_2, training=True)

            # Normalizamos los vectores
            z_1 = self.normalize(z_1)
            z_2 = self.normalize(z_2)

            # Classifier
            y_1 = self.classifier(z_1, training=True)
            y_2 = self.classifier(z_2, training=True)

            # Loss
            loss = self.compute_loss(z_1, z_2, y_1, y_2, self.lamda, self.tau)

        # Actualizar
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(loss)
        return {'loss': self.loss_tracker.result()}

    def normalize(self, x):
        return (x-tf.reduce_mean(x, axis=0)) / (tf.math.reduce_std(x, axis=0) + 1e-8)

    def compute_loss(self, z_1, z_2, y_1, y_2, lamda, tau):
        batch_size, d = tf.shape(z_1)[0], tf.shape(z_1)[1]

        z_1 = tf.reshape(z_1, (batch_size, -1, 1))
        z_2 = tf.reshape(z_2, (batch_size, -1, 1))


        # matriz de correlación cruzada
        C = tf.matmul(z_1, z_2, transpose_b=True)
        C = tf.cast(C, tf.float32)

        # Matriz identidad
        dim = tf.shape(C)[1]
        I = tf.eye(num_rows=dim, num_columns=dim, batch_shape=[batch_size])


        # Loss Lm
        softmax_matrix = tf.nn.softmax(C / tau, axis=1)
        loss_Lm = tf.keras.losses.categorical_crossentropy(I, softmax_matrix, from_logits=False, axis=1)

        Loss_Ly = tf.reduce_mean(y_1*(1-y_1) + y_2*(1-y_2))

        loss = loss_Lm + lamda * Loss_Ly
        return loss

    def get_encoded_data(self, X):
        z = self.encoder.predict(X)
        z = self.normalize(z)
        return z


Creamos el modelo y entrenamos con todos los datos

In [14]:
model = ContrastiveLearning(input_shape=(32, 32, 3), num_classes=NUM_CLASES, lamda=0.5, tau=5)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt)

classifier = MiClasificador()

In [15]:
semisupervised_training(model, classifier, train_data, train_label, unlabelled_data)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 296s 187ms/step - loss: 7.5034
315/315 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.0214 - loss: 4.6269
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0360 - loss: 4.3976
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0530 - loss: 4.2606
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0592 - loss: 4.1524
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0740 - loss: 4.0984
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0778 - loss: 4.0093
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0892 - loss: 3.9773
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1015 - loss: 3.9224
Epoch 9/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1018 - loss: 3.8616
Epoch 10/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1151 - loss: 3.7961
Epoch 11/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4m

(<ContrastiveLearning name=contrastive_learning_1, built=False>,
 <__main__.MiClasificador at 0x7d3cfff69510>)

In [16]:
pred = model.get_encoded_data(x_test)
print('Test accuracy :', classifier.score(pred, y_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Test accuracy : 0.10220000147819519


Probamos con el conjunto con los datos atípicos eliminados

In [23]:
model = ContrastiveLearning(input_shape=(32, 32, 3), num_classes=NUM_CLASES, lamda=0.5, tau=0.1)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [24]:
semisupervised_training(model, classifier, train_data, train_label, unlabelled_data_sin_atipicos)

1434/1434 ━━━━━━━━━━━━━━━━━━━━ 271s 186ms/step - loss: 7.7651
315/315 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0184 - loss: 5.0159
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0279 - loss: 4.4732
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0343 - loss: 4.3890
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0328 - loss: 4.3563
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0367 - loss: 4.3298
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0390 - loss: 4.2953
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0458 - loss: 4.2723
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0519 - loss: 4.2445
Epoch 9/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0462 - loss: 4.2295
Epoch 10/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0531 - loss: 4.2135
Epoch 11/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms

(<ContrastiveLearning name=contrastive_learning_3, built=False>,
 <__main__.MiClasificador at 0x7d3cfff69510>)

In [25]:
pred = model.get_encoded_data(x_test)
print('Test accuracy :', classifier.score(pred, y_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Test accuracy : 0.04969999939203262
